In [ ]:
!pip install fake-useragent pandas requests pyperclip

## Asks for token address

In [ ]:
token_address = input("Enter the token address: ")

# Get the transfer data
Need change

In [ ]:
from fake_useragent import UserAgent
import requests

In [ ]:
def get_random_user_agent():
    ua = UserAgent()
    return ua.random

def download_csv(url, output_file):
    headers = {
        "User-Agent": get_random_user_agent()
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        with open(output_file, 'wb') as file:
            file.write(response.content)

        print(f"File downloaded successfully and saved as {output_file}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

url = f"https://api-v2.solscan.io/v2/token/transfer/export?address={token_address}"
output_file = "transfer_data.csv"

download_csv(url, output_file)


# Calculate the date range of the transactions

In [ ]:
import pandas as pd

In [ ]:
# Load data into a DataFrame
df = pd.read_csv('transfer_data.csv')

# Extract min and max time values
start = df['Time'].min()
end = df['Time'].max()


print(f"Min Time: {start}")
print(f"Max Time: {end}")

# Get the price data using the date range
User must open the link and copy the hole raw data. If user copies the same thing twice it doesn't detect the copy

In [ ]:
import pyperclip
import time
import json

def generate_link():
    link = f"https://gmgn.ai/api/v1/token_kline/sol/{token_address}?resolution=1s&from={start}&to={end}"
    print("Generated Link:")
    print(link)


def wait_for_copy():
    last_clipboard_content = pyperclip.paste()  # Get the current clipboard content

    print("Waiting for a copy action...")
    while True:
        time.sleep(0.5)  # Poll every 0.5 seconds
        current_clipboard_content = pyperclip.paste()
        if current_clipboard_content != last_clipboard_content:
            print("Copy action detected!")
            return current_clipboard_content

generate_link()
json_data = wait_for_copy()
# segunda opcion
# json_data = input("Paste the copied JSON data here: ")

json_data = json.loads(json_data)

data_list = json_data['data']['list']

df = pd.DataFrame(data_list)
df.rename(columns={"time": "Time"}, inplace=True)

# Change the time column to be the first column
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

# Remove the volume column
df.drop(columns=['volume'], inplace=True)

# save the data into price_data.csv
df.to_csv('price_data.csv', index=False)

# Filter the transfers data (WIP)

In [ ]:
# Load the CSV files
csv1 = pd.read_csv("transfer_data.csv")
csv2 = pd.read_csv("transfer_data_to_delte.csv")

# Drop rows from csv1 that match any row in csv2
csv1_filtered = pd.merge(csv1, csv2, on=list(csv2.columns), how='left', indicator=True)
csv1_filtered = csv1_filtered[csv1_filtered['_merge'] == 'left_only'].drop(columns=['_merge'])

csv1_filtered['Time'] = csv1_filtered['Time'] + 3600

# Save the filtered result to a new CSV file
csv1_filtered.to_csv("filtered_transfer_data.csv", index=False)

print("Filtered CSV saved as 'filtered_transfer_data.csv'")